In [1]:
import guardrails as gd
import openai
from rich import print as rprint
import tiktoken
import dotenv
import os

from app.guardrails_specs.specs import rail_str

dotenv.load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPEN_API_KEY


## Checking if basic guardrails calls work

In [6]:
# Create a Guard class
guard = gd.Guard.from_rail_string(rail_str)

test_transaction = '''
Date
	
Transaction Description
	
Debit
	
Credit
	Balance
Jul 22, 2023	AMZ*Sleepsheep Direct		$141.75	$4,838.90
Jul 22, 2023	FIVE GUYS BURGERS & FR	$71.12		$4,980.65
Jul 22, 2023	ABC*ANYTIME FITNESS	$28.34		$4,909.53
Jul 21, 2023	RING MULTICAM MTH PLAN	$15.75		$4,881.19
Jul 19, 2023	DOORDASH*LUNA MEXICAN	$130.82		$4,865.44
Jul 18, 2023	PAYMENT - THANK YOU		$300.00	$4,734.62
Jul 16, 2023	SOLARA RESORT AND SPA	$63.00		$5,034.62
Jul 16, 2023	EDMONTO*3CINTEGRA PARKI	$87.00		$4,971.62
Jul 16, 2023	AMZN Mktp CA*TU5VJ3VX3	$16.73		$4,884.62
Jul 16, 2023	Amazon.ca*J29G10HJ3	$123.90		$4,867.89
Jul 16, 2023	ENTERPRISE CANADA C5E2	$428.22		$4,743.99
'''

# Wrap openai API call

raw_llm_output, guardrail_output = guard(
    openai.ChatCompletion.create,
    prompt_params={"transaction_string": test_transaction},
    
    model="gpt-3.5-turbo",
    max_tokens=2000,
    temperature=0.0,
    num_reasks=0,
)

rprint(guardrail_output)


/Users/fernandomeira/anaconda3/envs/llm_budget/lib/python3.11/site-packages/eliot/json.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if isinstance(o, (numpy.bool, numpy.bool_)):


{
    'transaction_list': [
        {
            'transaction_info': {
                'transaction_date': '2023-07-22',
                'transaction_description': 'AMZ*Sleepsheep Direct',
                'debit': 141.75,
                'credit': 0,
                'balance': 4838.9
            }
        },
        {
            'transaction_info': {
                'transaction_date': '2023-07-22',
                'transaction_description': 'FIVE GUYS BURGERS & FR',
                'debit': 71.12,
                'credit': 0,
                'balance': 4980.65
            }
        },
        {
            'transaction_info': {
                'transaction_date': '2023-07-22',
                'transaction_description': 'ABC*ANYTIME FITNESS',
                'debit': 28.34,
                'credit': 0,
                'balance': 4909.53
            }
        },
        {
            'transaction_info': {
                'transaction_date': '2023-07-21',
                'transaction_description': 'RING MULTICAM MTH PLAN',
                'debit': 15.75,
                'credit': 0,
                'balance': 4881.19
            }
        },
        {
            'transaction_info': {
                'transaction_date': '2023-07-19',
                'transaction_description': 'DOORDASH*LUNA MEXICAN',
                'debit': 130.82,
                'credit': 0,
                'balance': 4865.44
            }
        },
        {
            'transaction_info': {
                'transaction_date': '2023-07-18',
                'transaction_description': 'PAYMENT - THANK YOU',
                'debit': 300,
                'credit': 0,
                'balance': 4734.62
            }
        },
        {
            'transaction_info': {
                'transaction_date': '2023-07-16',
                'transaction_description': 'SOLARA RESORT AND SPA',
                'debit': 63,
                'credit': 0,
                'balance': 5034.62
            }
        },
        {
            'transaction_info': {
                'transaction_date': '2023-07-16',
                'transaction_description': 'EDMONTO*3CINTEGRA PARKI',
                'debit': 87,
                'credit': 0,
                'balance': 4971.62
            }
        },
        {
            'transaction_info': {
                'transaction_date': '2023-07-16',
                'transaction_description': 'AMZN Mktp CA*TU5VJ3VX3',
                'debit': 16.73,
                'credit': 0,
                'balance': 4884.62
            }
        },
        {
            'transaction_info': {
                'transaction_date': '2023-07-16',
                'transaction_description': 'Amazon.ca*J29G10HJ3',
                'debit': 123.9,
                'credit': 0,
                'balance': 4867.89
            }
        },
        {
            'transaction_info': {
                'transaction_date': '2023-07-16',
                'transaction_description': 'ENTERPRISE CANADA C5E2',
                'debit': 428.22,
                'credit': 0,
                'balance': 4743.99
            }
        }
    ]
}